# 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

- 1行に1記事の情報がJSON形式で格納される
- 各行には記事名が"title"キーに，記事本文が"text"キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
- ファイル全体はgzipで圧縮される  
  
以下の処理を行うプログラムを作成せよ．

In [2]:
import re

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [3]:
import gzip
import json

def load(filename, title):
    with gzip.open(filename, "rt", "utf_8") as text:
        lines = text.readline()
        while lines:
            elements = json.loads(lines)
            if elements["title"] == title:
                return elements["text"]
            lines = text.readline()
    return ""

In [ ]:
print(load("jawiki-country.json.gz", 'イギリス'))

---
## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [7]:
text = load("jawiki-country.json.gz", 'イギリス')
pattern = re.compile(r'\[\[Category:.+\]\]')
extraction = pattern.finditer( text)
for line in extraction:
    print(line.group())

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


---
## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [9]:
text = load("jawiki-country.json.gz", 'イギリス')
pattern = re.compile(r'\[\[Category:(.+)\]\]')
extraction = pattern.finditer( text)
for line in extraction:
    line = print(line.group(1))

イギリス|*
英連邦王国|*
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国|くれいとふりてん
1801年に設立された州・地域


---
## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [201]:
text = load("jawiki-country.json.gz", 'イギリス')
pattern = re.compile(r'=+.+=')
extraction = pattern.finditer(text)
for l in extraction:
    name = l.group().replace("=", "")
    level = int(l.group().count("=")/2-1)
    print(f"name:{name} \t level:{level}")

name:国名 	 level:1
name:歴史 	 level:1
name:地理 	 level:1
name:気候 	 level:2
name:政治 	 level:1
name:外交と軍事 	 level:1
name:地方行政区分 	 level:1
name:主要都市 	 level:2
name:科学技術 	 level:1
name:経済 	 level:1
name:鉱業 	 level:2
name:農業 	 level:2
name:貿易 	 level:2
name:通貨 	 level:2
name:企業 	 level:2
name:交通 	 level:1
name:道路 	 level:2
name:鉄道 	 level:2
name:海運 	 level:2
name:航空 	 level:2
name:通信 	 level:1
name:国民 	 level:1
name:言語 	 level:2
name:宗教 	 level:2
name: 婚姻  	 level:2
name:教育 	 level:2
name:文化 	 level:1
name:食文化 	 level:2
name:文学 	 level:2
name: 哲学  	 level:2
name:音楽 	 level:2
name:イギリスのポピュラー音楽 	 level:3
name:映画 	 level:2
name:コメディ 	 level:2
name:国花 	 level:2
name:世界遺産 	 level:2
name:祝祭日 	 level:2
name:スポーツ 	 level:1
name:サッカー 	 level:2
name:競馬 	 level:2
name:モータースポーツ 	 level:2
name:脚注 	 level:1
name:関連項目 	 level:1
name:外部リンク 	 level:1


---
## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [17]:
text = load("jawiki-country.json.gz", 'イギリス')
pattern = re.compile(r'^.*ファイル:.+$|^.*File:.+$',  re.MULTILINE)
extraction = pattern.finditer( text)
for l in extraction:
    print(re.sub(r'^.*ファイル:|^.*File:|\|.*', "", l.group()))

Royal Coat of Arms of the United Kingdom.svg
Battle of Waterloo 1815.PNG
The British Empire.png
Uk topo en.jpg
BenNevis2005.jpg
Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.jpg
Palace of Westminster, London - Feb 2007.jpg
David Cameron and Barack Obama at the G20 Summit in Toronto.jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
Scotland Parliament Holyrood.jpg
London.bankofengland.arp.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow T5.jpg
Anglospeak.svg
CHANDOS3.jpg
The Fabs.JPG
PalaceOfWestminsterAtNight.jpg
Westminster Abbey - West Door.jpg
Edinburgh Cockburn St dsc06789.jpg
Canterbury Cathedral - Portal Nave Cross-spire.jpeg
Kew Gardens Palm House, London - July 2009.jpg
2005-06-27 - United Kingdom - England - London - Greenwich.jpg
Stonehenge2007 07 30.jpg
Yard2.jpg
Durham Kathedrale Nahaufnahme.jpg
Roman Baths in Bath Spa, England - July 2006.jpg
Fountains Abbey vie

---
## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [8]:
import pprint
text = load("jawiki-country.json.gz", 'イギリス')
pattern1 = re.compile(r'^\{\{基礎情報 国.*^\}\}$',  re.MULTILINE + re.DOTALL)
extraction = pattern1.finditer(text)
for temp in extraction:
    temp = temp.group()
    pattern2 = re.compile(r'^\|.+=.+$', re.MULTILINE)
    lines = pattern2.finditer(temp)
    dic = {line.group()[1:].split(" = ")[0]: line.group()[1:].split(" = ")[1] for line in lines}
pprint.pprint(dic)

{'GDP/人': '36,727<ref name="imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref '
          'name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= '
          'IMF>Data and Statistics>World Economic Outlook Databases>By '
          'Countrise>United Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB / GBR',
 'ccTLD': '[[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm '
        'United Nations Department of Economic and Social Affairs>Population '
        'Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像

---
## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [136]:
text = load("jawiki-country.json.gz", 'イギリス')
pattern1 = re.compile(r'^\{\{基礎情報 国.*^\}\}$',  re.MULTILINE + re.DOTALL)
extraction = pattern1.finditer(text)
for temp in extraction:
    temp = temp.group()
    pattern2 = re.compile(r'^\|.+=.+$', re.MULTILINE)
    lines = pattern2.finditer(temp)
    dic = {}
    for line in lines:
        pattern3 = re.compile(r"'{2,5}")
        line = pattern3.sub( "", line.group()[1:])
        dic[line.split(" = ")[0]] = line.split(" = ")[1]
        print(f'name:{line.split(" = ")[0]} \t value:{line.split(" = ")[1]}')
#print(dic)

name:略名 	 value:イギリス
name:日本語国名 	 value:グレートブリテン及び北アイルランド連合王国
name:公式国名 	 value:{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
name:国旗画像 	 value:Flag of the United Kingdom.svg
name:国章画像 	 value:[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
name:国章リンク 	 value:（[[イギリスの国章|国章]]）
name:標語 	 value:{{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）
name:国歌 	 value:[[女王陛下万歳|神よ女王陛下を守り給え]]
name:位置画像 	 value:Location_UK_EU_Europe_001.svg
name:公用語 	 value:[[英語]]（事実上）
name:首都 	 value:[[ロンドン]]
name:最大都市 	 value:ロンドン
name:元首等肩書 	 value:[[イギリスの君主|女王]]
name:元首等氏名 	 value:[[エリザベス2世]]
name:首相等肩書 	 value:[[イギリスの首相|首相]]
name:首相等氏名 	 value:[[デーヴィッド・キャメロン]]
name:面積順位 	 value:76
name:面積大きさ 	 value:1 E11
name:面積値 	 value:244,820
name:水面積率 	 value:1.3%
name:人口統計年 	 value:2011
name:人口順位 	 value:22
name:人口大きさ 	 value:1 E7
name:人口値 	 value:63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Socia

---
## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [137]:
text = load("jawiki-country.json.gz", 'イギリス')
pattern1 = re.compile(r'^\{\{基礎情報 国.*^\}\}$',  re.MULTILINE + re.DOTALL)
extraction = pattern1.finditer(text)
for temp in extraction:
    temp = temp.group()
    pattern2 = re.compile(r'^\|.+=.+$', re.MULTILINE)
    lines = pattern2.finditer(temp)
    dic = {}
    for line in lines:
        pattern3 = re.compile(r"\[\[ファイル.+\|.+\||\[\[.{1,10}\||\[\[|\]\]|'{2,5}")
        line = pattern3.sub( "", line.group()[1:])
        dic[line.split(" = ")[0]] = line.split(" = ")[1]
        print(f'name:{line.split(" = ")[0]} \t value:{line.split(" = ")[1]}')
#print(dic)

name:略名 	 value:イギリス
name:日本語国名 	 value:グレートブリテン及び北アイルランド連合王国
name:公式国名 	 value:{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
name:国旗画像 	 value:Flag of the United Kingdom.svg
name:国章画像 	 value:イギリスの国章
name:国章リンク 	 value:（国章）
name:標語 	 value:{{lang|fr|Dieu et mon droit}}<br/>（フランス語:神と私の権利）
name:国歌 	 value:神よ女王陛下を守り給え
name:位置画像 	 value:Location_UK_EU_Europe_001.svg
name:公用語 	 value:英語（事実上）
name:首都 	 value:ロンドン
name:最大都市 	 value:ロンドン
name:元首等肩書 	 value:女王
name:元首等氏名 	 value:エリザベス2世
name:首相等肩書 	 value:首相
name:首相等氏名 	 value:デーヴィッド・キャメロン
name:面積順位 	 value:76
name:面積大きさ 	 value:1 E11
name:面積値 	 value:244,820
name:水面積率 	 value:1.3%
name:人口統計年 	 value:2011
name:人口順位 	 value:22
name:人口大きさ 	 value:1 E7
name:人口値 	 value:63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>
name:人口密度値 	 value:246
name:GDP統計年元 	 value:2012
name:GD

---
## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [19]:
text = load("jawiki-country.json.gz", 'イギリス')
pattern1 = re.compile(r'^\{\{基礎情報 国.*^\}\}$',  re.MULTILINE + re.DOTALL)
extraction = pattern1.finditer(text)
for temp in extraction:
    temp = temp.group()
    pattern2 = re.compile(r'^\|.+=.+$', re.MULTILINE)
    lines = pattern2.finditer(temp)
    dic = {}
    for line in lines:
        pattern3 = re.compile(r"\[\[ファイル.+\|.+\||\[\[.{1,10}\||\[\[|\]\]|'{2,5}|<.+>|\{\{.+\||\}\}", re.MULTILINE)
        line = pattern3.sub("", line.group()[1:])
        dic[line.split(" = ")[0]] = line.split(" = ")[1]
        print(f'name:{line.split(" = ")[0]} \t value:{line.split(" = ")[1]}')
#print(dic)

name:略名 	 value:イギリス
name:日本語国名 	 value:グレートブリテン及び北アイルランド連合王国
name:公式国名 	 value:United Kingdom of Great Britain and Northern Ireland
name:国旗画像 	 value:Flag of the United Kingdom.svg
name:国章画像 	 value:イギリスの国章
name:国章リンク 	 value:（国章）
name:標語 	 value:Dieu et mon droit（フランス語:神と私の権利）
name:国歌 	 value:神よ女王陛下を守り給え
name:位置画像 	 value:Location_UK_EU_Europe_001.svg
name:公用語 	 value:英語（事実上）
name:首都 	 value:ロンドン
name:最大都市 	 value:ロンドン
name:元首等肩書 	 value:女王
name:元首等氏名 	 value:エリザベス2世
name:首相等肩書 	 value:首相
name:首相等氏名 	 value:デーヴィッド・キャメロン
name:面積順位 	 value:76
name:面積大きさ 	 value:1 E11
name:面積値 	 value:244,820
name:水面積率 	 value:1.3%
name:人口統計年 	 value:2011
name:人口順位 	 value:22
name:人口大きさ 	 value:1 E7
name:人口値 	 value:63,181,775
name:人口密度値 	 value:246
name:GDP統計年元 	 value:2012
name:GDP値元 	 value:1兆5478億
name:GDP統計年MER 	 value:2012
name:GDP順位MER 	 value:5
name:GDP値MER 	 value:2兆4337億
name:GDP統計年 	 value:2012
name:GDP順位 	 value:6
name:GDP値 	 value:2兆3162億
name:GDP/人 	 value:36,727
name:建国形態 	 value:建国
name:

---
## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [25]:
#28を実行して辞書dicを作成しておく

import sys
import requests

S = requests.Session()

URL = "https://www.mediawiki.org/w/api.php"

PARAMS = {
    "action":"query",
    "format":"json",
    "prop": "imageinfo",
    "titles": "File:{}".format(dic[u'国旗画像']),
    "iiprop" : "url"
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

image = DATA["query"]["pages"]["-1"]["imageinfo"][0]["url"]
print(image)

#print(json.dumps(DATA, indent=4))

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
